In [19]:
import pandas as pd
import os
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk import word_tokenize
from nltk.tag import CRFTagger

In [20]:
df_anis = pd.read_csv('https://raw.github.com/ghazafm/SocialMediaSentiment/main/preprocessing/Training/data/raw/anis.csv')
df_prabowo = pd.read_csv('https://raw.github.com/ghazafm/SocialMediaSentiment/main/preprocessing/Training/data/raw/prabowo.csv')
df_ganjar = pd.read_csv('https://raw.github.com/ghazafm/SocialMediaSentiment/main/preprocessing/Training/data/raw/ganjar.csv')

df_anis.drop('Unnamed: 0',axis=1,inplace=True)
df_prabowo.drop('Unnamed: 0',axis=1,inplace=True)
df_ganjar.drop('Unnamed: 0',axis=1,inplace=True)

In [21]:
df_anis.drop_duplicates(inplace=True)
df_prabowo.drop_duplicates(inplace=True)
df_ganjar.drop_duplicates(inplace=True)
df_anis.dropna(inplace=True)
df_prabowo.dropna(inplace=True)
df_ganjar.dropna(inplace=True)

In [22]:
def remove_str_index(word,token):
    temp = [0]
    for j in word:    
        if not j == temp[-1]:
            temp.append(j)
    temp.remove(0)
    temp = ''.join(temp)
    return temp


def clear_double(data,token=False):
    temp = []
    data = data.split()
    for word in data:
        temp.append(remove_str_index(word,token))    
    if token:
        return temp
    else:
        return ' '.join(temp)

In [23]:
df_anis['no_double'] = df_anis['Tweet'].apply(clear_double)
df_prabowo['no_double'] = df_prabowo['Tweet'].apply(clear_double)
df_ganjar['no_double'] = df_ganjar['Tweet'].apply(clear_double)

In [24]:
alay = pd.read_csv('https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv')

In [25]:
alay = dict(zip(alay['slang'], alay['formal']))

In [26]:
def cek_alay(word, alay):
    return alay.get(word, word)


def clear_alay(data):
    words = str(data)
    words = words.split()
    cleared_words = [cek_alay(word, alay) for word in words]
    return ' '.join(cleared_words)

In [27]:
df_anis['no_alay'] = df_anis['no_double'].apply(clear_alay)
df_prabowo['no_alay'] = df_prabowo['no_double'].apply(clear_alay)
df_ganjar['no_alay'] = df_ganjar['no_double'].apply(clear_alay)

In [28]:
ct = CRFTagger()
ct.set_model_file('all_indo_man_tag_corpus_model.crf.tagger')

In [29]:
def tokenizer(text):
    text = word_tokenize(text)
    return text

In [30]:
double_meaning = [
    'jadi', 'menjadi', 'bapak', 'kalau', 'rakyat', 'siapa', 
    'apa', 'orang', 'bakal', 'sama', 'pasang', 'jelang', 'tahun', 'hari', 
    'bersama', 'mau', 'tetap', 'buat', 'for', 'bukan', 'semua', 
    'terus', 'si', 'inilah', 'kan', 'tak', 'banyak', 'meski', 'lebih', 'keputusan', 
    'final', 'paling', 'hasil', 'umum', 'tepat', 'tersebut', 'total', 'klik', 'capres', 
    'pilih', 'pemilihan', 'terpilih', 'survei', 'survey', 'pemilu', 'terkait', 'fahnoor', 
    'nan', 'calon', 'pilpres', 'resmi', 'cocok', 'politik', 'ribuan', 'ratusan', 'nama','maju',
    'hut', 'dapat', 'semoga', 'beliau', 'besar', 'makin', 'layak', 'partai', 'mendukung', 'dukung', 
    'dukungan', 'gubernur', 'masyarakat', 'warga','presiden','ri','inismyname','pilpres','nan','calon','indonesia','survei','survey','pemilu'
]

name = [
    'inismyname', 'indonesia', 'rosiade', 'joko', 'jokowi', 'widodo', 'ridwan', 'kamil', 'rosiade', 
    'thohir', 'mujani', 'erick', 'saiful', 'chotimah', 'ahy', 'bukan', 'aniesahy', 'ahmad', 
    'pks', 'pdip', 'jawa', 'puan', 'maharani', 'pan', 'jateng', 'tengah', 'megawati', 'ppp', 
    'rasyid', 'gerindra', 'nasdem', 'demokrat', 'pkb', 'allah', #maaffff,
    'anis', 'anies', 'baswedan', 'prabowo', 'subianto', 'ganjar', 'pranowo','fahnoor', 'amien','sandiaga',
    'chotimah', 'uno'
]

def clean_manual(data):
    data = str(data)
    data = data.split()
    data = [word for word in data if not word in double_meaning]
    data = [word for word in data if not word in name]
    data = ' '.join(data)
    return data

In [31]:
df_anis['clean_manual'] = df_anis['no_alay'].apply(clean_manual)
df_prabowo['clean_manual'] = df_prabowo['no_alay'].apply(clean_manual)
df_ganjar['clean_manual'] = df_ganjar['no_alay'].apply(clean_manual)

In [32]:
df_anis

,Tweet,label,no_double,no_alay,clean_manual
0,info anies presiden,Positive,info anies presiden,info anies presiden,info
1,politisi partai gerindra sandiaga uno menjawab...,Positive,politisi partai gerindra sandiaga uno menjawab...,politisi partai gerindra sandiaga uno menjawab...,politisi menjawab soal diduetkan kembali denga...
2,lanjut pak anies kita kawal sampai jadi presiden,Positive,lanjut pak anies kita kawal sampai jadi presiden,lanjut pak anies kita kawal sampai jadi presiden,lanjut pak kita kawal sampai
3,semoga allah swt menyelamatkan bangsa dan nega...,Positive,semoga alah swt menyelamatkan bangsa dan negar...,semoga alah swt menyelamatkan bangsa dan negar...,alah swt menyelamatkan bangsa dan negara repub...
4,chotimah kasian ya pa anies makanya sudah teka...,Positive,chotimah kasian ya pa anies makanya sudah teka...,chotimah kasihan ya apa anies makanya sudah te...,kasihan ya makanya sudah tekad keluarga saya
...,...,...,...,...,...
9995,tidak ada gejolak sara selama membangun pks pu...,Negative,tidak ada gejolak sara selama membangun pks pu...,tidak ada gejolak sara selama membangun pks pu...,tidak ada gejolak sara selama membangun puji c...
9996,ubedilah mahfud md otak di balik perppu ciptak...,Negative,ubedilah mahfud md otak di balik perpu ciptake...,ubedilah mahfud md otak di balik perpu ciptake...,ubedilah mahfud md otak di balik perpu ciptake...
9997,my presiden mranies,Negative,my presiden mranies,my presiden mranies,my mranies
9998,pa anies presiden,Negative,pa anies presiden,apa anies presiden,


In [33]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
def stemming(data):
   data = stemmer.stem(data)
   return data

In [34]:
df_anis['stemmed'] = df_anis['clean_manual'].apply(stemming)
df_prabowo['stemmed'] = df_prabowo['clean_manual'].apply(stemming)
df_ganjar['stemmed'] = df_ganjar['clean_manual'].apply(stemming)

In [35]:
df_anis

,Tweet,label,no_double,no_alay,clean_manual,stemmed
0,info anies presiden,Positive,info anies presiden,info anies presiden,info,info
1,politisi partai gerindra sandiaga uno menjawab...,Positive,politisi partai gerindra sandiaga uno menjawab...,politisi partai gerindra sandiaga uno menjawab...,politisi menjawab soal diduetkan kembali denga...,politis jawab soal duet kembali dengan mantan ...
2,lanjut pak anies kita kawal sampai jadi presiden,Positive,lanjut pak anies kita kawal sampai jadi presiden,lanjut pak anies kita kawal sampai jadi presiden,lanjut pak kita kawal sampai,lanjut pak kita kawal sampai
3,semoga allah swt menyelamatkan bangsa dan nega...,Positive,semoga alah swt menyelamatkan bangsa dan negar...,semoga alah swt menyelamatkan bangsa dan negar...,alah swt menyelamatkan bangsa dan negara repub...,alah swt selamat bangsa dan negara republik da...
4,chotimah kasian ya pa anies makanya sudah teka...,Positive,chotimah kasian ya pa anies makanya sudah teka...,chotimah kasihan ya apa anies makanya sudah te...,kasihan ya makanya sudah tekad keluarga saya,kasihan ya makanya sudah tekad keluarga saya
...,...,...,...,...,...,...
9995,tidak ada gejolak sara selama membangun pks pu...,Negative,tidak ada gejolak sara selama membangun pks pu...,tidak ada gejolak sara selama membangun pks pu...,tidak ada gejolak sara selama membangun puji c...,tidak ada gejolak sara lama bangun puji covid ...
9996,ubedilah mahfud md otak di balik perppu ciptak...,Negative,ubedilah mahfud md otak di balik perpu ciptake...,ubedilah mahfud md otak di balik perpu ciptake...,ubedilah mahfud md otak di balik perpu ciptake...,ubedilah mahfud md otak di balik perpu ciptake...
9997,my presiden mranies,Negative,my presiden mranies,my presiden mranies,my mranies,my mranies
9998,pa anies presiden,Negative,pa anies presiden,apa anies presiden,,


In [36]:
stopword = StopWordRemoverFactory().create_stop_word_remover()

def clear_stopwords(data):
    data = str(data)
    data = stopword.remove(data)
    return data

In [37]:
df_anis['no_stopwords'] = df_anis['stemmed'].apply(clear_stopwords)
df_prabowo['no_stopwords'] = df_prabowo['stemmed'].apply(clear_stopwords)
df_ganjar['no_stopwords'] = df_ganjar['stemmed'].apply(clear_stopwords)

In [38]:
df_anis

,Tweet,label,no_double,no_alay,clean_manual,stemmed,no_stopwords
0,info anies presiden,Positive,info anies presiden,info anies presiden,info,info,info
1,politisi partai gerindra sandiaga uno menjawab...,Positive,politisi partai gerindra sandiaga uno menjawab...,politisi partai gerindra sandiaga uno menjawab...,politisi menjawab soal diduetkan kembali denga...,politis jawab soal duet kembali dengan mantan ...,politis jawab soal duet dengan mantan dki jaka...
2,lanjut pak anies kita kawal sampai jadi presiden,Positive,lanjut pak anies kita kawal sampai jadi presiden,lanjut pak anies kita kawal sampai jadi presiden,lanjut pak kita kawal sampai,lanjut pak kita kawal sampai,lanjut pak kawal
3,semoga allah swt menyelamatkan bangsa dan nega...,Positive,semoga alah swt menyelamatkan bangsa dan negar...,semoga alah swt menyelamatkan bangsa dan negar...,alah swt menyelamatkan bangsa dan negara repub...,alah swt selamat bangsa dan negara republik da...,alah swt selamat bangsa negara republik para p...
4,chotimah kasian ya pa anies makanya sudah teka...,Positive,chotimah kasian ya pa anies makanya sudah teka...,chotimah kasihan ya apa anies makanya sudah te...,kasihan ya makanya sudah tekad keluarga saya,kasihan ya makanya sudah tekad keluarga saya,kasihan makanya tekad keluarga
...,...,...,...,...,...,...,...
9995,tidak ada gejolak sara selama membangun pks pu...,Negative,tidak ada gejolak sara selama membangun pks pu...,tidak ada gejolak sara selama membangun pks pu...,tidak ada gejolak sara selama membangun puji c...,tidak ada gejolak sara lama bangun puji covid ...,gejolak sara lama bangun puji covid kamis janu...
9996,ubedilah mahfud md otak di balik perppu ciptak...,Negative,ubedilah mahfud md otak di balik perpu ciptake...,ubedilah mahfud md otak di balik perpu ciptake...,ubedilah mahfud md otak di balik perpu ciptake...,ubedilah mahfud md otak di balik perpu ciptake...,ubedilah mahfud md otak balik perpu ciptaker k...
9997,my presiden mranies,Negative,my presiden mranies,my presiden mranies,my mranies,my mranies,my mranies
9998,pa anies presiden,Negative,pa anies presiden,apa anies presiden,,,


In [39]:
df_anis.dropna(inplace=True)
df_prabowo.dropna(inplace=True)
df_ganjar.dropna(inplace=True)

In [40]:
df_anis

,Tweet,label,no_double,no_alay,clean_manual,stemmed,no_stopwords
0,info anies presiden,Positive,info anies presiden,info anies presiden,info,info,info
1,politisi partai gerindra sandiaga uno menjawab...,Positive,politisi partai gerindra sandiaga uno menjawab...,politisi partai gerindra sandiaga uno menjawab...,politisi menjawab soal diduetkan kembali denga...,politis jawab soal duet kembali dengan mantan ...,politis jawab soal duet dengan mantan dki jaka...
2,lanjut pak anies kita kawal sampai jadi presiden,Positive,lanjut pak anies kita kawal sampai jadi presiden,lanjut pak anies kita kawal sampai jadi presiden,lanjut pak kita kawal sampai,lanjut pak kita kawal sampai,lanjut pak kawal
3,semoga allah swt menyelamatkan bangsa dan nega...,Positive,semoga alah swt menyelamatkan bangsa dan negar...,semoga alah swt menyelamatkan bangsa dan negar...,alah swt menyelamatkan bangsa dan negara repub...,alah swt selamat bangsa dan negara republik da...,alah swt selamat bangsa negara republik para p...
4,chotimah kasian ya pa anies makanya sudah teka...,Positive,chotimah kasian ya pa anies makanya sudah teka...,chotimah kasihan ya apa anies makanya sudah te...,kasihan ya makanya sudah tekad keluarga saya,kasihan ya makanya sudah tekad keluarga saya,kasihan makanya tekad keluarga
...,...,...,...,...,...,...,...
9995,tidak ada gejolak sara selama membangun pks pu...,Negative,tidak ada gejolak sara selama membangun pks pu...,tidak ada gejolak sara selama membangun pks pu...,tidak ada gejolak sara selama membangun puji c...,tidak ada gejolak sara lama bangun puji covid ...,gejolak sara lama bangun puji covid kamis janu...
9996,ubedilah mahfud md otak di balik perppu ciptak...,Negative,ubedilah mahfud md otak di balik perpu ciptake...,ubedilah mahfud md otak di balik perpu ciptake...,ubedilah mahfud md otak di balik perpu ciptake...,ubedilah mahfud md otak di balik perpu ciptake...,ubedilah mahfud md otak balik perpu ciptaker k...
9997,my presiden mranies,Negative,my presiden mranies,my presiden mranies,my mranies,my mranies,my mranies
9998,pa anies presiden,Negative,pa anies presiden,apa anies presiden,,,


In [41]:
df_anis = df_anis[['Tweet','no_double','no_alay','clean_manual','stemmed','no_stopwords','label']]
df_prabowo = df_prabowo[['Tweet','no_double','no_alay','clean_manual','stemmed','no_stopwords','label']]
df_ganjar = df_ganjar[['Tweet','no_double','no_alay','clean_manual','stemmed','no_stopwords','label']]

In [42]:
concat = pd.concat([df_anis,df_prabowo,df_ganjar])

In [43]:
dir = os.path.abspath(os.path.join('..', 'data/clean/regular'))
df_anis.to_csv(f'{dir}/anis.csv',index=False)
df_prabowo.to_csv(f'{dir}/prabowo.csv',index=False)
df_ganjar.to_csv(f'{dir}/ganjar.csv',index=False)
concat.to_csv(f'{dir}/gabungan.csv',index=False)